In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd 
data_pd = pd.read_csv('/space/hotel/hieud/mlflow_aisia/img_without_woman_coats.csv')
train_val_pd , test_pd = train_test_split(data_pd, test_size=0.1, random_state=42, stratify=data_pd['category'])
train_pd , val_pd = train_test_split(train_val_pd, test_size=0.2, random_state=42, stratify=train_val_pd['category'])

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import cv2

# Function to extract color histogram features
def extract_color_histogram(image_path):
    # Load image
    image = cv2.imread(image_path)
    
    # Convert image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Compute color histograms for each channel
    hist_r = cv2.calcHist([image_rgb], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image_rgb], [1], None, [256], [0, 256])
    hist_b = cv2.calcHist([image_rgb], [2], None, [256], [0, 256])
    
    # Normalize histograms
    hist_r = hist_r.flatten() / np.sum(hist_r)
    hist_g = hist_g.flatten() / np.sum(hist_g)
    hist_b = hist_b.flatten() / np.sum(hist_b)
    
    # Concatenate histograms
    features = np.concatenate((hist_r, hist_g, hist_b))
    
    return features

# List of image paths
image_paths = train_val_pd['img_path']  # Replace with your image paths

# Extract features for each image
features_list = [extract_color_histogram(img_path) for img_path in image_paths]

# Convert features to numpy array
features_array = np.array(features_list)

# Apply t-SNE for dimensionality reduction
tsne = TSNE(n_components=2, random_state=42)
embedded_features = tsne.fit_transform(features_array)

# Plot t-SNE embedding
index_path_dict = {}
for index, path in enumerate(image_paths):
    index_path_dict[str(index)] = path

In [17]:
print(index_path_dict)

{'0': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/women_coats/img_22828453.jpg', '1': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/dresses/img_44325965.jpg', '2': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/women_tshirts/img_58890277.jpg', '3': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/women_pants/img_58890186.jpg', '4': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/dresses/img_21408221.jpg', '5': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/men_trousers/img_49251896.jpg', '6': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/men_trousers/img_50323342.jpg', '7': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/men_sweaters/img_42234762.jpg', '8': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/men_trousers/img_58495490.jpg', '9': '/space/hotel/hieud/mlflow_aisia/image/AISIA_BOUTIQUE_DATASET/women_sweaters/img_50443424.jpg', '10': '/space/hote

In [29]:
from scipy.spatial.distance import euclidean

def precision_at_K(query_image_path, top_5_recommend):
    y_cat = np.array([query_image_path.split('/')[-2]]*5)
    # y_cat = np.array(['cardigans']*5)
    y_pred = np.array([cat.split('/')[-2] for cat in top_5_recommend])
    comparison_array = (y_cat == y_pred)
    true_positives = np.sum(comparison_array)
    print(y_cat, y_pred)
    return true_positives/(true_positives+  len(comparison_array)-true_positives)
def find_similar_images(query_image_index, embedded_features):
    query_feature = embedded_features[query_image_index]
    distances = [euclidean(query_feature, feature) for feature in embedded_features]
    sorted_indices = np.argsort(distances)
    top_5_recommend = sorted_indices[:5]
    top_5_recommend_cat = [index_path_dict[str(index)] for index in top_5_recommend]
    
    print(precision_at_K(index_path_dict[str(query_image_index)], top_5_recommend_cat))
    

# Find similar images for the first image (index 0)
similar_indices = find_similar_images(0, embedded_features)

# # Display similar images
# for i in similar_indices:
#     img_path = image_paths.iloc[i]
#     image = cv2.imread(img_path)
    
#     # Convert image from BGR to RGB
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
#     plt.imshow(image_rgb)
#     plt.title(img_path)
#     plt.show()
    

['women_coats' 'women_coats' 'women_coats' 'women_coats' 'women_coats'] ['women_coats' 'men_trousers' 'men_trousers' 'dresses' 'women_sweaters']
0.2
